In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gymnasium as gym
import numpy as np
from tensoraerospace.agent import PPO


env = gym.make('Pendulum-v1')

class ActionNormalizer(gym.ActionWrapper):
    """Rescale and relocate the actions."""

    def action(self, action: np.ndarray) -> np.ndarray:
        """Change the range (-1, 1) to (low, high)."""
        low = self.action_space.low
        high = self.action_space.high

        scale_factor = (high - low) / 2
        reloc_factor = high - scale_factor

        action = action * scale_factor + reloc_factor
        action = np.clip(action, low, high)

        return action

    def reverse_action(self, action: np.ndarray) -> np.ndarray:
        """Change the range (low, high) to (-1, 1)."""
        low = self.action_space.low
        high = self.action_space.high

        scale_factor = (high - low) / 2
        reloc_factor = high - scale_factor

        action = (action - reloc_factor) / scale_factor
        action = np.clip(action, -1.0, 1.0)


        return action

In [2]:

env = ActionNormalizer(env)
env.reset()

agent = PPO(env, gamma=0.9, max_episodes = 100)

agent.train()

100%|██████████| 100/100 [02:57<00:00,  1.77s/it]


In [3]:
agent.save()

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.ref_signal to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.ref_signal` for environment variables or `env.get_wrapper_attr('ref_signal')` that will search the reminding wrappers.
  logger.warn(


In [4]:
agent.publish_to_hub("Mr8bit/ppo-pendelium",
                     "./Apr29_04-21-29_PPO",
                     access_token="")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

critic.pth: 100%|██████████| 4.10k/4.10k [00:01<00:00, 2.80kB/s]
actor.pth: 100%|██████████| 5.66k/5.66k [00:01<00:00, 3.85kB/s]
Upload 2 LFS files: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]


In [6]:
from tensoraerospace.agent import PPO

agent = PPO.from_pretrained("./Apr29_04-21-29_PPO")

In [2]:
from tensoraerospace.agent import PPO

agent = PPO.from_pretrained(repo_name="Mr8bit/ppo-pendelium",
                            access_token="")

Mr8bit/ppo-pendelium


Fetching 4 files: 100%|██████████| 4/4 [00:02<00:00,  1.64it/s]
